# Objet : Analyse des relations entre champs des données IRVE

## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par la détection d'incohérences de relations
- l'analyse a posteriori permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 2% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Cohérence date maj stations](#6---Cohérence-date-maj-stations)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d erreurs résiduelles](#8---Exemples-d-erreurs-résiduelles)
- [Annexe : Initialisation du modèle de données 'mermaid'](#Annexe-:-Initialisation-du-modèle-de-données-'mermaid')

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [3]:
# la variable 'irve' est définie en fin de Notebook
mermaid(irve)

In [217]:
def analyse_integrite(data):
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [311]:
from collections import Counter
from observation import Ilist, Iindex, Analysis
import pandas as pd
res = {}
chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'
res['date'] = '2023-03-03'
file = 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv'
irve = pd.read_csv(chemin + file, sep=',', low_memory=False)
res['len_irve'] = len(irve)
print('nombre de lignes : ', res['len_irve']) 

nombre de lignes :  51276


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [281]:
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

In [224]:
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
champs =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- un peu moins de 1 % des points de charge sont hors itinerance

In [351]:
data = irve.loc[:, champs]
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].loc[:, champs].reset_index()['index']
itinerance = data[~data['non_concerne']].loc[:, champs].reset_index()
itinerance = itinerance.loc[:, relations]
print('nombre de pdc hors itinerance : ', len(non_concerne))
print('nombre de pdc en itinerance   : ', len(itinerance))

nombre de pdc hors itinerance :  607
nombre de pdc en itinerance   :  50669


-----------------------
## 4 - Bilan initial intégrité
- 36 % des lignes sont erronées ( 18 116 )

In [314]:
res = analyse_integrite(itinerance)
itinerance_ok_1 = itinerance.loc[itinerance.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance.loc[~itinerance.ok, relations].reset_index(drop=True)
itinerance = itinerance.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_1))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_1))
print("taux d'erreur : ", round(len(itinerance_1) / len(itinerance) * 100), ' %')

index - id_pdc_itinerance                          16123
contact_operateur - id_station_itinerance          10719
nom_enseigne - id_station_itinerance                7514
coordonneesXY - id_station_itinerance              11825
id_station_itinerance - id_pdc_itinerance            578
nom_station - id_station_itinerance                 1865
implantation_station - id_station_itinerance        1245
nbre_pdc - id_station_itinerance                    1458
condition_acces - id_station_itinerance               35
horaires - id_station_itinerance                    9869
station_deux_roues - id_station_itinerance         10968
adresse_station - coordonneesXY                     1360

nombre d'enregistrements sans erreurs :  32553
nombre d'enregistrements avec au moins une erreur :  18116
taux d'erreur :  36  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de diviser par 5 le nombre de lignes erronnées ( 2 166 )

In [336]:
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by='date_maj').duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
print('nombre de doublons pdc : ', len(doublons_pdc))
print('nombre de pdc sans doublon   : ', len(itinerance_2))

nombre de doublons pdc :  8301
nombre de pdc sans doublon   :  9815


In [337]:
res = analyse_integrite(itinerance_2)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_3))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_3))

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            200
nom_enseigne - id_station_itinerance                 457
coordonneesXY - id_station_itinerance                624
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  204
implantation_station - id_station_itinerance         136
nbre_pdc - id_station_itinerance                     513
condition_acces - id_station_itinerance                5
horaires - id_station_itinerance                     194
station_deux_roues - id_station_itinerance           521
adresse_station - coordonneesXY                      587

nombre d'enregistrements sans erreurs :  7649
nombre d'enregistrements avec au moins une erreur :  2166


-----------------------
## 6 - Cohérence date maj stations
- 20% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de 25% le nombre de lignes erronnées ( 1 616 )
- les dernières erreurs correspondent à 468 stations associées à 32 opérateurs et sont liées à des causes multiples

In [338]:
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
print('nombre de doublons stations : ', len(doublons_stat_maj))
print('nombre de pdc sans doublon   : ', len(itinerance_4))

nombre de doublons stations :  375
nombre de pdc sans doublon   :  1791


In [339]:
res = analyse_integrite(itinerance_4)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_5))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_5))

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                 267
coordonneesXY - id_station_itinerance                216
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                   40
implantation_station - id_station_itinerance          23
nbre_pdc - id_station_itinerance                     452
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      53
station_deux_roues - id_station_itinerance           325
adresse_station - coordonneesXY                      414

nombre d'enregistrements sans erreurs :  175
nombre d'enregistrements avec au moins une erreur :  1616


In [340]:
il = Ilist.obj(itinerance_5)
print(il.tree())

-1: root-derived (1616)
   0 : index (1616)
      2 : nom_enseigne (317)
         9 : condition_acces (2)
      3 : coordonneesXY (413)
      4 : adresse_station (468)
         1 : contact_operateur (32)
      5 : id_station_itinerance (468)
         13: date_maj (82)
      6 : nom_station (432)
      7 : implantation_station (4)
      8 : nbre_pdc (21)
      10: horaires (49)
      11: station_deux_roues (5)
      12: id_pdc_itinerance (1616)
      14: last_modified (54)


In [324]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  12


-----------------------
## 7 - Synthèse


In [345]:
#itinerance_5_full.to_csv('erreurs_IRVE.csv')

itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
print('total des doublons à supprimer : ', itinerance['doublons_a_supprimer'].sum())

total des lignes à corriger :  1616
total des doublons à supprimer :  8676


In [349]:
itinerance_ok = itinerance[~itinerance['lignes_ko']].loc[:, relations].reset_index(drop=True)
print('nombre de pdc : ', len(itinerance_ok))
print('\nbilan intégrité :')
res = analyse_integrite(itinerance_ok)

nombre de pdc :  40377

bilan intégrité :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                  0
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                    0
implantation_station - id_station_itinerance           0
nbre_pdc - id_station_itinerance                       0
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                        0


-----------------------
## 8 - Exemples d erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 12 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [310]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1536,47980,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la BoÃ©tie 75008 Paris,FRV75PPX0811,Paris | Rue la BoÃ©tie 37,Voirie,5,Accès libre,24/7,True,FRV75EPX08112,2022-12-16,2023-03-03T17:35:31.881000
1537,47981,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la BoÃ©tie 75008 Paris,FRV75PPX0811,Paris | Rue la BoÃ©tie 37,Voirie,5,Accès libre,24/7,True,FRV75EPX08111,2022-12-16,2023-03-03T17:35:31.881000
1538,47982,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la BoÃ©tie 75008 Paris,FRV75PPX0811,Paris | Rue la BoÃ©tie 37,Voirie,5,Accès libre,24/7,False,FRV75EPX08114,2022-12-16,2023-03-03T17:35:31.881000
1539,47983,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la BoÃ©tie 75008 Paris,FRV75PPX0811,Paris | Rue la BoÃ©tie 37,Voirie,5,Accès libre,24/7,False,FRV75EPX08113,2022-12-16,2023-03-03T17:35:31.881000
1540,47984,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la BoÃ©tie 75008 Paris,FRV75PPX0811,Paris | Rue la BoÃ©tie 37,Voirie,5,Accès libre,24/7,False,FRV75EPX08115,2022-12-16,2023-03-03T17:35:31.881000
1541,47985,assistance.technique@tevgo.fr,Belib',"[2.314663, 48.8737]",37 Rue la Boétie 75008 Paris,FRV75PPX0811,Paris | Rue la Boétie 37,Voirie,6,Accès libre,24/7,False,FRV75EPX08116,2022-12-16,2023-03-03T17:35:31.881000
1648,49154,assistance.technique@tevgo.fr,Belib',"[2.360503, 48.86502]",18 Rue PerrÃ©e 75003 Paris,FRV75PPX0304,Paris | Rue PerrÃ©e 18,Voirie,5,Accès libre,24/7,False,FRV75EPX03041,2022-12-16,2023-03-03T17:35:31.881000
1649,49155,assistance.technique@tevgo.fr,Belib',"[2.360503, 48.86502]",18 Rue PerrÃ©e 75003 Paris,FRV75PPX0304,Paris | Rue PerrÃ©e 18,Voirie,5,Accès libre,24/7,False,FRV75EPX03042,2022-12-16,2023-03-03T17:35:31.881000
1650,49156,assistance.technique@tevgo.fr,Belib',"[2.360503, 48.86502]",18 Rue PerrÃ©e 75003 Paris,FRV75PPX0304,Paris | Rue PerrÃ©e 18,Voirie,5,Accès libre,24/7,False,FRV75EPX03043,2022-12-16,2023-03-03T17:35:31.881000
1651,49157,assistance.technique@tevgo.fr,Belib',"[2.360503, 48.86502]",18 Rue PerrÃ©e 75003 Paris,FRV75PPX0304,Paris | Rue PerrÃ©e 18,Voirie,5,Accès libre,24/7,False,FRV75EPX03044,2022-12-16,2023-03-03T17:35:31.881000


### Cohérence implantation_station - id_station
- 23 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [302]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1363,43183,contact@qovoltis.com,QOVOLTIS,"[-2.7649063, 48.4941535]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE201290,2022-07-25,2023-03-03T17:36:05.518000
1364,43184,contact@qovoltis.com,QOVOLTIS,"[-2.7649063, 48.4941535]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE201426,2022-07-25,2023-03-03T17:36:05.518000
1365,43185,contact@qovoltis.com,QOVOLTIS,"[-2.7649063, 48.4941535]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE201396,2022-07-25,2023-03-03T17:36:05.518000
1366,43186,contact@qovoltis.com,QOVOLTIS,"[-0.69173, 44.83914]","19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 MER...",FRQOVP3300015,"19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 ME",Parking privé à usage public,6,Accès libre,Mo-Fr 09:00-19:00,0,FRQOVE207049,2022-05-24,2023-03-03T17:36:05.518000
1367,43187,contact@qovoltis.com,QOVOLTIS,"[-0.69173, 44.83914]","19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 MER...",FRQOVP3300015,"19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 ME",Parking privé à usage public,6,Accès libre,Mo-Fr 09:00-19:00,0,FRQOVE206440,2022-05-24,2023-03-03T17:36:05.518000
1368,43188,contact@qovoltis.com,QOVOLTIS,"[-0.69173, 44.83914]","19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 MER...",FRQOVP3300015,"19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 ME",Parking privé à usage public,6,Accès libre,Mo-Fr 09:00-19:00,0,FRQOVE207495,2022-05-24,2023-03-03T17:36:05.518000
1369,43190,contact@qovoltis.com,QOVOLTIS,"[-0.69173, 44.83914]","19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 MER...",FRQOVP3300015,"19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 ME",Parking privé à usage public,6,Accès libre,Mo-Fr 09:00-19:00,0,FRQOVE203131,2022-05-24,2023-03-03T17:36:05.518000
1372,43282,contact@qovoltis.com,QOVOLTIS,"[-0.69173, 44.83914]","19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 MER...",FRQOVP3300015,"19 AVENUE NEIL ARMSTRONG, Le Caucase 33700 ME",Parking privé à usage public,6,Accès libre,Mo-Fr 09:00-19:00,0,FRQOVE203131TE,2022-05-24,2023-03-03T17:36:05.518000
1373,43283,contact@qovoltis.com,QOVOLTIS,"[-2.7649063, 48.4941535]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205871TE,2022-07-25,2023-03-03T17:36:05.518000
1375,43343,contact@qovoltis.com,QOVOLTIS,"[1.8088319, 43.6986226]",1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,FRQOVP8100001,1 AVENUE GEORGES POMPIDOU 81500 LAVAUR,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205471,2022-05-10,2023-03-03T17:36:05.518000


### Cohérence nom_station - id_station
- 40 pdc sont associés à une station avec un nom non cohérent

In [301]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
81,1752,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 3,Parking privé à usage public,2,Accès libre,24/7,false,FRS86ESOREB1402,2020-04-30,2023-03-03T17:40:12.327000
82,1753,service-recharge@soregies.fr,ALTERBASE86,"[0.37178, 46.56869]","78, avenue Jacques Coeur - 86000 - Poitiers",FRS86PSOREB140,Sorégies 2,Parking privé à usage public,2,Accès libre,24/7,false,FRS86ESOREB1401,2020-04-30,2023-03-03T17:40:12.327000
107,2862,support@lastmilesolutions.com,Last Mile Solutions,"[5.585043, 47.149426]",Aire de Dole Audelange - Autoroute A36 - 39700...,FRLMSP1000123210,APRR DOLE ROMANGE,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRLMSE10001232101,2023-02-15,2023-03-03T17:39:38.719000
108,2864,support@lastmilesolutions.com,Last Mile Solutions,"[5.585043, 47.149426]",Aire de Dole Audelange - Autoroute A36 - 39700...,FRLMSP1000123211,APRR DOLE ROMANGE,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRLMSE10001232111,2023-02-15,2023-03-03T17:39:38.719000
109,2877,support@lastmilesolutions.com,Last Mile Solutions,"[5.585043, 47.149426]",Aire de Dole Audelange - Autoroute A36 - 39700...,FRLMSP1000123211,APRR DOLE AUDELANGE,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRLMSE10001232112,2023-02-15,2023-03-03T17:39:38.719000
110,2878,support@lastmilesolutions.com,Last Mile Solutions,"[5.585043, 47.149426]",Aire de Dole Audelange - Autoroute A36 - 39700...,FRLMSP1000123210,APRR DOLE AUDELANGE,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRLMSE10001232102,2023-02-15,2023-03-03T17:39:38.719000
254,6412,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550405,2022-12-15,2023-03-03T17:39:32.426000
256,6431,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2022-12-15,2023-03-03T17:39:32.426000
257,6435,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2022-12-15,2023-03-03T17:39:32.426000
310,7672,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2022-12-15,2023-03-03T17:39:32.426000


### Cohérence adresse - coordonnées
- 414 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [300]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,10,info@rossinienergy.com,RossiniEnergy,"[3.262326, 49.856113]","4 Avenue Archimède, 02100 Saint-Quentin",FRROSE352,Aventim_PC_1,Parking privé à usage public,10,Accès libre,24/7,FALSE,FRROSE3521,2022-07-12,2023-03-03T17:43:34.993000
1,11,info@rossinienergy.com,RossiniEnergy,"[3.262326, 49.856113]","7 Avenue Archimède, 02100 Saint-Quentin",FRROSE352,Aventim_PC_1,Parking privé à usage public,10,Accès libre,24/7,FALSE,FRROSE3524,2022-07-12,2023-03-03T17:43:34.993000
2,12,info@rossinienergy.com,RossiniEnergy,"[3.262326, 49.856113]","6 Avenue Archimède, 02100 Saint-Quentin",FRROSE352,Aventim_PC_1,Parking privé à usage public,10,Accès libre,24/7,FALSE,FRROSE3523,2022-07-12,2023-03-03T17:43:34.993000
3,13,info@rossinienergy.com,RossiniEnergy,"[3.262326, 49.856113]","5 Avenue Archimède, 02100 Saint-Quentin",FRROSE352,Aventim_PC_1,Parking privé à usage public,10,Accès libre,24/7,FALSE,FRROSE3522,2022-07-12,2023-03-03T17:43:34.993000
4,17,info@rossinienergy.com,RossiniEnergy,"[3.262326, 49.856113]","9 Avenue Archimède, 02100 Saint-Quentin",FRROSE351,Aventim_PC_2,Parking privé à usage public,10,Accès libre,24/7,FALSE,FRROSE3512,2022-07-12,2023-03-03T17:43:34.993000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,50861,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-03-03T17:35:31.203000
1787,50862,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-03-03T17:35:31.203000
1788,50863,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-03-03T17:35:31.203000
1789,50864,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-03-03T17:35:31.203000


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [298]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[0.193942, 49.544211]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
83,1866,contact@ubitricity.com,Ubitricity France Network,"[0.193942, 49.544211]",Place Abbé Pierre,FRUBIE10056568,Centre ville - Montivilliers,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10056568,2023-01-18,2023-03-03T17:40:10.619000
1615,51263,contact@ubitricity.com,Ubitricity France Network,"[0.193942, 49.544211]",Place Abb� Pierre,FRUBIE10058428,Centre ville - Montivilliers,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10058428,2023-01-18,2023-03-03T16:22:10.908000


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [297]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1611,50861,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-03-03T17:35:31.203000
1612,50862,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-03-03T17:35:31.203000
1613,50863,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-03-03T17:35:31.203000
1614,50864,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-03-03T17:35:31.203000


### Cohérence station - coordonnées
- 216 pdc sont associés à des stations avec plusieurs adresses

In [299]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
83,1769,service-recharge@soregies.fr,ALTERBASE86,"[0.84278, 46.67931]",Place de l'etoile - 86260 - Saint-Pierre de Ma...,FRS86PSOREB091,Saint-Pierre de Maillé,Voirie,2,Accès libre,24/7,false,FRS86ESOREB0911,2017-01-31,2023-03-03T17:40:12.327000
84,1782,service-recharge@soregies.fr,ALTERBASE86,"[0.842456, 46.679573]",Place de l'etoile - 86260 - Saint-Pierre de Ma...,FRS86PSOREB091,Saint-Pierre de Maillé,Voirie,2,Accès libre,24/7,false,FRS86ESOREB0912,2017-01-31,2023-03-03T17:40:12.327000
254,6412,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550405,2022-12-15,2023-03-03T17:39:32.426000
256,6431,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2022-12-15,2023-03-03T17:39:32.426000
257,6435,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 Toulouse,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2022-12-15,2023-03-03T17:39:32.426000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,48430,assistance.technique@tevgo.fr,Belib',"[2.3036203, 48.869644]",248 Rue Pierre Charron 75008 Paris,FRV75PPX0801,Paris | Rue Pierre Charron 62,Voirie,3,Accès libre,24/7,False,FRV75EPX08012,2022-12-16,2023-03-03T17:35:31.881000
1575,48431,assistance.technique@tevgo.fr,Belib',"[2.3036203, 48.8696448]",248 Rue Pierre Charron 75008 Paris,FRV75PPX0801,Paris | Rue Pierre Charron 62,Voirie,4,Accès libre,24/7,False,FRV75EPX08014,2022-12-16,2023-03-03T17:35:31.881000
1616,48506,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07044,2022-12-16,2023-03-03T17:35:31.881000
1617,48507,assistance.technique@tevgo.fr,Belib',"[2.3077697, 48.8492973]",330 Avenue de Ségur 75007 Paris,FRV75PPX0704,Paris | Avenue de Ségur 55,Voirie,6,Accès libre,24/7,False,FRV75EPX07041,2022-12-16,2023-03-03T17:35:31.881000


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [292]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1169,42830,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-03-03T17:36:21.752000
1170,42831,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-03-03T17:36:21.752000
1171,42832,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EU6RQ5270,2022-07-21,2023-03-03T17:36:21.752000
1172,42833,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-03-03T17:36:21.752000
1173,42834,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-03-03T17:36:21.752000
1174,42835,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EUHNJ5886,2022-07-21,2023-03-03T17:36:21.752000
1175,42836,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-03-03T17:36:21.752000
1176,42837,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EXINB5826,2022-07-21,2023-03-03T17:36:21.752000
1179,42840,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-03-03T17:36:21.752000
1180,42841,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-03-03T17:36:21.752000


- exemple : une station avec plusieurs coordonnées -> erreur de saisie

In [289]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FRV75PPX0704', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1384,48380,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07042,2022-12-16,2023-03-03T17:35:31.881000
1399,48414,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07045,2022-12-16,2023-03-03T17:35:31.881000
1400,48415,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07046,2022-12-16,2023-03-03T17:35:31.881000
1445,48506,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07044,2022-12-16,2023-03-03T17:35:31.881000
1446,48507,assistance.technique@tevgo.fr,Belib',"[2.3077697, 48.8492973]",330 Avenue de Ségur 75007 Paris,FRV75PPX0704,Paris | Avenue de Ségur 55,Voirie,6,Accès libre,24/7,False,FRV75EPX07041,2022-12-16,2023-03-03T17:35:31.881000
1447,48509,assistance.technique@tevgo.fr,Belib',"[2.3077698, 48.849297]",330 Avenue de SÃ©gur 75007 Paris,FRV75PPX0704,Paris | Avenue de SÃ©gur 55,Voirie,5,Accès libre,24/7,False,FRV75EPX07043,2022-12-16,2023-03-03T17:35:31.881000


- exemple : une station par point de charge avec nbre_pdc = 1

In [254]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.191899, 49.506711]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
7119,51257,contact@ubitricity.com,Ubitricity France Network,"[0.191899, 49.506711]",Avenue Pr�sident Ren� Coty (Gare Beaulieu),FRUBIE10072897,Beaulieu,Voirie,1,Accès libre,24/7,FALSE,FRUBIE10072897,2023-01-18,2023-03-03T16:22:10.908000
7123,51261,contact@ubitricity.com,Ubitricity France Network,"[0.191899, 49.506711]",Avenue Pr�sident Ren� Coty (Gare Beaulieu),FRUBIE10091696,Beaulieu,Voirie,1,Accès libre,24/7,FALSE,FRUBIE10091696,2023-01-18,2023-03-03T16:22:10.908000


-----------------------
# Annexe : Initialisation du modèle de données 'mermaid'

In [1]:
from base64 import b64encode
from IPython.display import Image, display

def mermaid(graph):
  display(Image(url="https://mermaid.ink/img/" + b64encode(graph.encode("ascii")).decode("ascii")))

In [2]:
# modèle de données suivant l'application 'mermaid' 
irve = """ 
erDiagram
    AMENAGEUR ||..|{ STATION : amenage
    AMENAGEUR {
        string nom_amenageur
        string siren_amenageur
        string contact_amenageur 
    }
    OPERATEUR ||..|{ STATION : "exploite pour le compte de l enseigne"
    OPERATEUR {
        string contact_operateur PK "M"
        string nom_operateur 
        string telephone_operateur 
    }
    ENSEIGNE ||..|{ STATION : "heberge"
    ENSEIGNE {
        string nom_enseigne PK "M" 
    }
    STATION {
        string  id_station_itinerance PK "M"
        string  nom_station "M"
        enum    implantation_station "M"
        integer nbre_pdc "M"
        string  condition_acces "M"
        string  horaires "M"
        boolean station_deux_roues "M"
        string  id_station_local
        enum    raccordement
        string  num_pdl
        date    date_mise_en_service 
    }
    LOCALISATION ||--|{ STATION : "localise"
    LOCALISATION {
       array   coordonneesXY PK "M"
       string  adresse_station "M"
       string  code_insee_commune 
    }
    STATION ||--|{ POINT_DE_CHARGE : regroupe
    POINT_DE_CHARGE {
        string id_pdc_itinerance PK "M Root"
        number puissance_nominale "M"
        boolean prise_type_ef "M"
        boolean prise_type_2 "M"
        boolean prise_type_combo_ccs "M"
        boolean prise_type_chademo "M"
        boolean prise_type_autre "M"
        boolean paiement_acte "M"
        boolean paiement_autre "M"
        boolean reservation "M"
        enum    accessibilite_pmr "M"
        string  restriction_gabarit "M"
        date    date_maj "M"
        string  id_pdc_local
        boolean gratuit
        boolean paiement_cb
        string  tarification
        string  observations
        boolean cable_t2_attache 
    }
"""